In [ ]:
install.packages("coloc")

In [ ]:
install.packages("rlang")

In [ ]:
library(coloc)

In [ ]:
# Set the disease information
# This section is not automated due to the large size of the files
# Be sure to replace 'disease' with the file of interest
disease <- "CoronaryAtherosclerosis"
disease_file <- "CoronaryAtherosclerosis.tsv"
sample_size <- 282871
case_control_prop <- 0.271
eQTL_files <- list.files(path="eQTL_subsets", pattern='ENSG.*', full.names = TRUE)

In [ ]:
error_file_path <- "coloc_error_log.txt"
result_matrix <- list()

# Read the GWAS data for the first disease
# Same applies here, replace the file with the file of interest
curr_gwas <- na.omit(read.csv(paste("CoronaryAtherosclerosis.tsv", sep = ""), 
                              sep = '\t', header = TRUE,
                              colClasses = c("character", "double", "NULL", "double", "NULL",
                                             "NULL", "double", "NULL", "NULL", "NULL",
                                             "NULL", "NULL", "NULL", "NULL", "NULL",
                                             "NULL", "NULL", "NULL", "NULL")))

In [ ]:
# Remove duplicate SNPs
curr_gwas <- curr_gwas[!duplicated(curr_gwas[, "variant_id"]), ]

In [ ]:
print(nrow(curr_gwas))

In [ ]:
# Genome-wide significance threshold
p_value_threshold <- 5e-2

# Filter the dataframe
filtered_gwas <- curr_gwas[curr_gwas$p_value < p_value_threshold, ]

In [ ]:
print(nrow(filtered_gwas))

In [ ]:
# Prepare the GWAS list
gwas_list <- list()
gwas_list$MAF <- filtered_gwas$MAF_calculated_from_dosage_data
gwas_list$snp <- filtered_gwas$variant_id
gwas_list$position <- filtered_gwas$base_pair_location
gwas_list$N <- sample_size
gwas_list$pvalues <- filtered_gwas$p_value
gwas_list$type <- "cc"
gwas_list$s <- case_control_prop

In [ ]:
results_df <- data.frame(
  gene_name = character(),
  nsnps = numeric(),
  PP_H0_abf = numeric(),
  PP_H1_abf = numeric(),
  PP_H2_abf = numeric(),
  PP_H3_abf = numeric(),
  PP_H4_abf = numeric(),
  stringsAsFactors = FALSE
)

In [ ]:
# Parameters for running in batches. Starting at 1 and ending with 15000 runs all of the genes for that disease.
# Last run: 
batch_start <- 1
batch_end <- 15001

In [ ]:
# Code for executing the Colocalization Analysis. Process can take several hours for each disease, so be wary.
# Check the GWAS dataset
gwas_fail <- tryCatch({
    check_dataset(gwas_list)
}, error = function(cond) {
    error_message <- paste("Error: disease file ", disease, " has error: \n ", cond, sep = "")
    cat(error_message, file = error_file_path, append = TRUE)
})

if (inherits(gwas_fail, "error")) {
    print("Error in GWAS dataset. Skipping further processing.")
} else {
    curr_results <- list()
    
    # Iterate over eQTL files
    for (i in batch_start:batch_end) {
        if(i > length(eQTL_files)) {
            break 
          }
        
        eQTL_file <- eQTL_files[i]
        gene_name <- gsub(".csv$", "", basename(eQTL_file))  # Extract gene name from file name
        eQTL_data <- read.csv(eQTL_file)

        eQTL_list <- list()
        eQTL_list$beta <- eQTL_data$beta
        eQTL_list$varbeta <- eQTL_data$varbeta
        eQTL_list$snp <- eQTL_data$snp
        eQTL_list$position <- eQTL_data$pos
        eQTL_list$type <- eQTL_data[, 'type'][1]
        eQTL_list$N <- eQTL_data$N[0]
        eQTL_list$MAF <- eQTL_data$MAF

        # Check the eQTL dataset
        eQTL_fail <- tryCatch({
            suppressWarnings(check_dataset(eQTL_list))
        }, error = function(cond) {
            error_message <- paste("Error: eQTL file ", eQTL_file, " has error: \n ", cond$message, sep = "")
            cat(error_message, file = error_file_path, append = TRUE)
            return(NULL)  # Return NULL to indicate failure
        })

        if (inherits(eQTL_fail, "error")) {
            next
        }
        
        # Check for missing values in the "type" column
        if (any(is.na(eQTL_data$type))) {
            cat("Skipping processing for", eQTL_file, "due to missing values in 'type' column.\n")
            next 
        }
        
        #THIS IS WHERE I FILTER THE DATASETS
        common_snps <- intersect(gwas_list$snp, eQTL_list$snp)

        # Filter both gwas_list and eQTL_list to only include these common SNPs
        # And ensure they are in the same order

        gwas_list_filtered <- list(
          MAF = gwas_list$MAF[match(common_snps, gwas_list$snp)],
          snp = common_snps,  # This ensures the order matches
          position = gwas_list$position[match(common_snps, gwas_list$snp)],
          N = gwas_list$N,
          pvalues = gwas_list$pvalues[match(common_snps, gwas_list$snp)],
          type = gwas_list$type,
          s = gwas_list$s
        )

        # For eQTL_list - this part goes inside your loop where you prepare eQTL_list
        eQTL_list_filtered <- list(
          beta = eQTL_list$beta[match(common_snps, eQTL_list$snp)],
          varbeta = eQTL_list$varbeta[match(common_snps, eQTL_list$snp)],
          snp = common_snps,  
          position = eQTL_list$position[match(common_snps, eQTL_list$snp)],
          type = eQTL_list$type,
          N = eQTL_list$N,
          MAF = eQTL_list$MAF[match(common_snps, eQTL_list$snp)]
        )
        
        coloc_results <- tryCatch({
            coloc.abf(gwas_list_filtered, eQTL_list_filtered)
        }, error = function(err) {
            cat("Error in coloc.abf:", conditionMessage(err), "\n")
            return(NULL)
        })

        # Check if coloc_results is NULL
        if (is.null(coloc_results)) {
            next
        }
        
        temp_results <- data.frame(
          gene_name = gene_name,
          nsnps = coloc_results$summary["nsnps"],
          PP_H0_abf = coloc_results$summary["PP.H0.abf"],
          PP_H1_abf = coloc_results$summary["PP.H1.abf"],
          PP_H2_abf = coloc_results$summary["PP.H2.abf"],
          PP_H3_abf = coloc_results$summary["PP.H3.abf"],
          PP_H4_abf = coloc_results$summary["PP.H4.abf"],
          stringsAsFactors = FALSE
        )
        
        results_df <- rbind(results_df, temp_results)
        
    }
}

In [ ]:
row.names(results_df) <- NULL
View(results_df)

In [ ]:
if (!requireNamespace("dplyr", quietly = TRUE)) install.packages("dplyr")
library(dplyr)

# Only unique rows based on gene_name
results_df_unique <- results_df %>% distinct(gene_name, .keep_all = TRUE)

head(results_df_unique)
print(nrow(results_df_unique))

In [ ]:
# Run this to save the data from memory into a file
file_path <- "results_df_unique.csv"

if(file.exists(file_path)) {
  write.table(results_df_unique, file = file_path, sep = ",", row.names = FALSE, col.names = FALSE, append = TRUE, quote = TRUE)
} else {
  write.table(results_df_unique, file = file_path, sep = ",", row.names = FALSE, col.names = TRUE, append = FALSE, quote = TRUE)
}

print('done')

In [ ]:
# No need to run this -  only for testing purposes and reconfiguring batch if process dies
file_path <- "results_df.csv"

if(file.exists(file_path)) {
  write.table(results_df, file = file_path, sep = ",", row.names = FALSE, col.names = FALSE, append = TRUE, quote = TRUE)
} else {
  write.table(results_df, file = file_path, sep = ",", row.names = FALSE, col.names = TRUE, append = FALSE, quote = TRUE)
}

In [ ]:
results_df <- read.csv("results_df.csv")

head(results_df)

In [ ]:
# Unique gene names
unique_genes <- unique(results_df$gene_name)

print(length(unique_genes))